# Preprocessing MORED Dataset 

In [1]:
import os
from pathlib import Path


In [2]:
os.chdir('../..')


In [17]:
#pwd to avoid errors in data loading paths and other required paths ...

In [4]:
#Create or ensure the existence of the Folder (repository) containing the output figures:

os.makedirs("Figures/Preprocessing_Mored_Dataset", exist_ok=True)


In [13]:
#Data source path
source_Data = Path("Raw_Data/")
WP_Data_Path = source_Data/"WP_données"

In [14]:
#Path validity check
assert WP_Data_Path.is_dir(),"The data can be found on the public website: https://moredataset.github.io/MORED/ "


In [18]:
import numpy
print(numpy.__version__)

1.24.3


In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from tqdm.autonotebook import tqdm
np.random.seed()
tqdm.pandas()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16016\454157297.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [27]:
#first simple check :

df = pd.read_csv(WP_Data_Path/'APT1.csv')

df.head()

,timestamp,Vrms,real_power
0,18/04/2020 16:18:50,227.94,113
1,18/04/2020 16:19:00,228.43,114
2,18/04/2020 16:19:10,228.55,113
3,18/04/2020 16:19:20,228.07,114
4,18/04/2020 16:19:30,228.22,114


In [28]:
del df # then delete 

In [29]:

min_timestamp = None
max_timestamp = None

for f in tqdm(WP_Data_Path.glob("*.csv"), desc="Search for the minimum and maximum timestamp among premises ..."):
    with open(f, 'r') as file:
        header = next(file)
        first_line = next(file)
        timestamp = pd.to_datetime(first_line.split(',')[0], dayfirst=True)
        
        if min_timestamp is None or timestamp < min_timestamp:
            min_timestamp = timestamp
        
        if max_timestamp is None or timestamp > max_timestamp:
            max_timestamp = timestamp

print(f"Minimum timestamp for all residential premises: {min_timestamp}")
print(f"Maximum timestamp for all residential premises: {max_timestamp}")


Search for the minimum and maximum timestamp among premises ...: 0it [00:00, ?it/s]

Minimum timestamp for all residential premises: 2020-04-18 13:49:20
Maximum timestamp for all residential premises: 2021-08-15 10:48:00


# Reading data sets 

The various data sets will be read and pre-processed using different python libraries

### Brief description of the MORED dataset 
Since spring 2019, a data acquisition campaign has been underway to collect data reflecting the electricity consumption of various urban premises in different Moroccan cities. MORED is the first open African dataset on building electricity consumption. It contains labeled WP and IL consumption data, labeled IL signatures and WP consumption data from several Moroccan households and loads. The aim of this dataset is to continue the progress that the field of energy disaggregation has experienced over the past decade, by providing a dataset that contains more data and makes use of recent advances in the field.
First the data will be used for forecasting purposes ... then, after completion of the project, the part relating to disaggregation will also be studied.

###### MORED offers the following consumption data:
 * WP(whole power) and IL goround-truth electricity consumption
 * WP(whole power) electricity consumption  ( the main output of each single premise)
 * IL signatures ( individual load ) 

## WP electricity consumption 

In [142]:
# let's check the data from the first house in WP group , and inspect data and what we can do about it.

APT1 = pd.read_csv(WP_Data_Path/"APT1.csv", parse_dates = ['timestamp'],dayfirst = True)

APT1.head()

,timestamp,Vrms,real_power
0,2020-04-18 16:18:50,227.94,113
1,2020-04-18 16:19:00,228.43,114
2,2020-04-18 16:19:10,228.55,113
3,2020-04-18 16:19:20,228.07,114
4,2020-04-18 16:19:30,228.22,114


In [143]:
APT1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352485 entries, 0 to 352484
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   timestamp   352485 non-null  datetime64[ns]
 1   Vrms        352485 non-null  float64       
 2   real_power  352485 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 8.1 MB



First of all, the single APT1 series takes up a substantial 8.1 MB of space, which needs to be optimized for storage purposes.
Second we can see that python interpreted 'real_power' column as int64 , maybe because pandas did infer this 
data type when reading the csv ,this shows that an assumption has been made in regard of the column that maybe
stores mainly whole numbers ( i'll keep it like that )

In [144]:
# memory usage: 8.1 MB !

def reduce_memory_footprint(df : pd.DataFrame) -> pd.DataFrame:
    """
    This function can be used to reduce the memory footprint of the dataframe by using the apropriate Type casting...

    """
    dtypes = df.dtypes
    flottantsCol = dtypes[dtypes == "float64"].index.tolist()
    entiers_Col = dtypes[dtypes == "int64"].index.tolist()
    df[flottantsCol] = df[flottantsCol].astype("float32")
    df[entiers_Col] = df[entiers_Col].astype("int32")
    return df


In [145]:
APT1 = reduce_memory_footprint(APT1)

In [146]:
APT1.info() # a reduction of 5.4 MB is what we can do at least 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352485 entries, 0 to 352484
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   timestamp   352485 non-null  datetime64[ns]
 1   Vrms        352485 non-null  float32       
 2   real_power  352485 non-null  int32         
dtypes: datetime64[ns](1), float32(1), int32(1)
memory usage: 5.4 MB


We note that the type casting to float or integer value didn't change that match , maybe the timestamps contributes heavily   

In [147]:
a,b,c =APT1.iloc[0]

In [148]:
from sys import getsizeof
a,b,c =APT1.iloc[0]
print(f'The size of each datatype in each column is:  \n\t Timestamp : {getsizeof(a)}  MB ,\n\t Vrms {getsizeof(b)}  MB ,\n\t Real Power {getsizeof(c)}  MB ')

The size of each datatype in each column is:  
	 Timestamp : 128  MB ,
	 Vrms 28  MB ,
	 Real Power 28  MB 


Now we can see clearly how much timestamp datatype can occupy !

In [62]:
APT1.isna().any() # No NA values

timestamp     False
Vrms          False
real_power    False
dtype: bool

<b> To better optimize our dataframe, which is a mathematical time series $\mathbb{TS}$ such that:</b> 

$ \hspace{0.5cm}$ A time series $\mathbb{TS}$: is a sequence of data points, in the form of pairs  of timestamps and values, arranged in ascending order in time $ \mathbb{TS} = \langle (t_1, v_1) ,( t_2, v_2 ), . .\rangle$ . For each pair
$(t_i, v_i), 1 ≤ i$, the timestamp $t_i$ represents the time when the value vi ∈ R was recorded. A time series $ \mathbb{TS} = \langle (t_1, v_1), .... (t_n, v_n) \rangle$ with a fixed number of n data points is a bounded time series.
Another interesting feature here is that its regular , in other words the points are evenly spaced in time .

<b>Segment</b>:
$\newline$
$ \hspace{1cm} $A segment :  .<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) is a 5-tuple $ S = (t_s, t_e, SI, G_{ts} :TSG \to 2^{t_s,t_s+SI,...,t_e} , m) $
that represent in compact way all the points in a TS, 
The 5-tuple consist of the start date (timestamp) $t_s$, the end date $t_e$, the sampling interval SI, a function $G_{ts}$ which  for the $\mathbb{TS}$ $\in$ $\mathbb{TSG}$ gives the set of 'Timestamps' (or date points) for wich $\upsilon$ = $\perp$ $\in$ TS, the values of all other 'Timestamps' are defined by the model m multiplied by a scaling constant $CT_S \in \mathbb{R}$.



<a name="cite_note-1"></a>1. [^](#cite_ref-1) Preprint of: S. K. Jensen, T. B. Pedersen, and C. Thomsen, “Scalable Model-Based Management of Correlated Dimensional Time Series in ModelarDB+,” IEEE 37th ICDE, 2021, pp. 1380-1391, doi: 10.1109/ICDE51399.2021.00123, Copyright IEEE

Considering the two definitions , we can arrive at an intuitive solution to the problem of timestamps being a heavy weight on time series storage.

In [101]:
def segment(data: pd.DataFrame, col: str) -> tuple:
    '''
    This function segments a time series data frame and extracts relevant information.
    Args:
        data (pd.DataFrame): The input DataFrame containing timestamp and value columns.
        col (str): The column name for the time series values.

    Returns:
        tuple: A tuple containing (start_time, ID, time_series, time_series_length)
    '''
    start_time = data['timestamp'].min()
    unique_premises = data['Premises'].iloc[0]
    # Sort the DataFrame by timestamp
    data.sort_values(by=['timestamp'], inplace=True)
    # Extract the time series values as a numpy array for better performance
    time_series = data[col].values
    time_series_length = len(time_series)
    return start_time, unique_premises, time_series, time_series_length

def ts_segment(dataframe: pd.DataFrame, freq="10S", ts_id="series_name", col_name="series_value") -> pd.DataFrame:
    '''
    This function segments a DataFrame by 'Premises', extracts time series, and creates a summary DataFrame.
    Args:
        dataframe (pd.DataFrame): The input DataFrame containing timestamp, Premises, and value columns.
        freq (str): The sampling rate string.
        ts_id (str): The name of the time series ID column in the output DataFrame.
        col_name (str): The name of the time series value column in the output DataFrame.

    Returns:
        pd.DataFrame: A summary DataFrame with segmented time series data.
    '''
    premises_groups = dataframe.groupby('Premises')
    summary_data = {
        ts_id: [],
        'start_time': [],
        'sampling_rate': freq,
        col_name: [],
        'ts_length': [],
    }

    for _, group_df in tqdm(premises_groups, leave=False):
        start, unique_premises, ts, ts_length = segment(group_df, col_name)
        summary_data[ts_id].append(unique_premises)
        summary_data['start_time'].append(start)
        summary_data[col_name].append(ts)  # Convert to list if needed if needed : ts.tolist()
        summary_data['ts_length'].append(ts_length)

    return pd.DataFrame(summary_data)

# Example usage:
# summary_df = ts_segment(data_frame, freq="10S", ts_id="series_name", col_name="series_value")


In [102]:
#let's see the impact on a TS wich APT1 timeserie:
TS = reduce_memory_footprint(APT1)

In [103]:
#add Premises col to identify the origin of the time serie, and other possible inputs in this case : 

TS['Premises'] = 'Premise_1'
col = 'real_power' # 'real_power


In [104]:
# Store the segmented form in another DataFrame:

TS_segmented = ts_segment(TS, freq="5S", ts_id="Mains", col_name=col)

In [106]:
TS_segmented

,Mains,start_time,sampling_rate,real_power,ts_length
0,Premise_1,2020-04-18 16:18:50,5S,"[113, 114, 113, 114, 114, 113, 114, 115, 113, ...",352485


In [109]:
TS_segmented.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Mains          1 non-null      object        
 1   start_time     1 non-null      datetime64[ns]
 2   sampling_rate  1 non-null      object        
 3   real_power     1 non-null      object        
 4   ts_length      1 non-null      int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 168.0+ bytes


In [120]:
TS_segmented.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Mains          1 non-null      object        
 1   start_time     1 non-null      datetime64[ns]
 2   sampling_rate  1 non-null      object        
 3   real_power     1 non-null      object        
 4   ts_length      1 non-null      int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 168.0+ bytes


In [149]:
percentage = ((TS_segmented.memory_usage(deep = True).sum()/1024) / (TS.memory_usage(deep = True).sum()/1024) ) *100
print(f'Optimization was successfully achieved at a rate of  : {100 - round(percentage,6)}% !!! \nAWESOME!')


Optimization was successfully achieved at a rate of  : 99.998654% !!! 
AWESOME!
